Configurar Google Translate API e baixar o json
https://cloud.google.com/translate/docs/basic/setup-basic

In [0]:
%mv ./Translation*.json ./key.json

Baixa MilkQA para tradução

In [1]:
!wget https://www.dropbox.com/s/1ozzofw3f7thq3g/MilkQA-v2.tar.gz
!tar -xzvf MilkQA-v2.tar.gz
%cd MilkQA-v2/data

Streaming output truncated to the last 5000 lines.
MilkQA-v2/data/5128-question.txt
MilkQA-v2/data/20103-question.txt
MilkQA-v2/data/15784-answer.txt
MilkQA-v2/data/7330-question.txt
MilkQA-v2/data/15965-answer.txt
MilkQA-v2/data/24612-answer.txt
MilkQA-v2/data/18130-question.txt
MilkQA-v2/data/7177-question.txt
MilkQA-v2/data/16998-answer.txt
MilkQA-v2/data/16904-question.txt
MilkQA-v2/data/25130-question.txt
MilkQA-v2/data/2772-question.txt
MilkQA-v2/data/2596-question.txt
MilkQA-v2/data/11772-answer.txt
MilkQA-v2/data/10203-answer.txt
MilkQA-v2/data/2181-question.txt
MilkQA-v2/data/4340-question.txt
MilkQA-v2/data/16135-question.txt
MilkQA-v2/data/1874-question.txt
MilkQA-v2/data/2916-question.txt
MilkQA-v2/data/6178-answer.txt
MilkQA-v2/data/2772-answer.txt
MilkQA-v2/data/17609-question.txt
MilkQA-v2/data/6-answer.txt
MilkQA-v2/data/6213-answer.txt
MilkQA-v2/data/221-question.txt
MilkQA-v2/data/25125-answer.txt
MilkQA-v2/data/12124-answer.txt
MilkQA-v2/data/14160-answer.txt
MilkQA-

Pegar as questões, assuntos e respostas e passar para lista de palavras a serem traduzidas

In [10]:
!pip install tqdm
import os
import re
from pathlib import Path
from tqdm import tqdm

files = os.listdir(os.curdir)
files = filter(lambda x: "question" in x, files)
numbers = [re.search("(.*)-", x).group(1) for x in files]

words = set()
unsolved = set()
for number in tqdm(numbers):
        question = Path(number + "-question.txt").read_text()
        question = re.split("<.*>", question)
        question = list(filter(None, question))
        
        if len(question) != 2:
            unsolved.add(number)
            continue

        words.add(question[0])
        words.add(question[1])

        answer = Path(number + "-answer.txt").read_text()
        words.add(answer)

100%|██████████| 2657/2657 [00:00<00:00, 13737.09it/s]


Imprimir os exemplos mal-formatados não resolvidos

In [0]:
print(unsolved)

['25414', '13050', '13546', '1487', '16762', '10551']


Resolver a questão 13546

In [0]:
words.add("cana de açucar")
words.add("""sou de dois vizinhos , e dou assitencia técnica para 
 produtores de leite da agricultura familiar desta cidade . E gostria
    de ter algumas informações sobre cana de açucar cal pecuario na
    suplementação de vacas de leite.""")
answer = Path("13546-answer.txt").read_text()
words.add(answer)

Resolver questões 13050, 10551 e 1487: sem assunto

In [4]:
for number in [13050,10551,1487]:
        print(number)
        question = Path(str(number) + "-question.txt").read_text()
        question = re.sub("<.*>", "", question)
        print(question)
        words.add("")
        words.add(question)

        answer = Path(str(number) + "-answer.txt").read_text()
        words.add(answer)

13050


QUAL É O PRIMEIRO PASSO PARA COMEÇAR UMA CRIAÇÃO DE VACA LEITERAS NO 
NORDESTE DA BAHIA?

10551


Gostaria de informações sobre o trato de vacas em lactação, com MELAÇO
de cana.A eficiencia do produto e se poderia utilizar com uréia+sulfato
de amônia.Qual a quantidade diária por animal?Devo utilizar o em pó
diluído em que proporção?

1487

Como combater a figueira nas vacas leiterias?



Resolver questões 25414, 16762: sem delimitador \<corpo\>

In [7]:
for number in [25414,16762]:
        print(number)
        question = Path(str(number) + "-question.txt").read_text()
        question = re.sub("<.*>", "", question)
        question = re.split("\n\n", question)
        question = list(filter(None, question))

        assert len(question) == 2

        words.add(question[0])
        words.add(question[1])
        
        answer = Path(str(number) + "-answer.txt").read_text()
        words.add(answer)

25414
16762


Importar biblioteca de tradução de lista de palavras

In [0]:
!jupyter nbconvert --to python Auxiliar.ipynb

In [0]:
import Auxiliar
Auxiliar.translate2dct(words, 'milkqa-dic.json')